In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import geopandas as gpd
import rtree
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely import wkt
import descartes

import fiona
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)

sns.set_palette(sns.diverging_palette(10, 220, sep=80, n=3, center="dark"))

sns.set_context('talk')

In [29]:
puds = gpd.read_file('../data/viz/puds.shp', crs = {'init' :'epsg:4326'})
puds

,OBJECTID,PUD_NAME,PUD_WEB_UR,PUD_CHANGE,PUD_ZONING,PUD_STATUS,PUD_CASE_N,SHAPEAREA,SHAPELEN,GLOBALID,WARD,year_scrap,year,geometry
0,75158,4L Associates Joint Venture,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=80-17,"Official Digital Zoning Map of July 1, 2010",R-5-A,Final,80-17,0,0,{95450152-37F7-41C5-B147-0C2719D9EAFA},Ward 5,1981,1981,"POLYGON ((-77.00471574325915 38.92462196572229, -77.00473300809519 38.92468044427013, -77.004756..."
1,75159,Bozutto Group,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=08-24C/04-25,PUD Modification July 2018,"C-2-B, R-5-B",Final,08-24C/04-25,0,0,{1FF4275D-29FF-4D38-8CFE-D7119010FAF5},Ward 5,2018,2018,"(POLYGON ((-76.99633034837495 38.9303615341334, -76.99688238904335 38.93036155015298, -76.998094..."
2,75160,National Academy of Sciences - Fifth Street,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=01-34,"Official Digital Zoning Map of July 1, 2010",C-3-C,Final,01-34,0,0,{F5A40354-52E1-486B-A268-201E28D938FF},Ward 2,2002,2002,"POLYGON ((-77.01907499933805 38.89721344569826, -77.01907474363878 38.89625726176668, -77.019730..."
3,75161,Baywood Hotels,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=11-25,Case # 11-25,C-3-C,Final,11-25,0,0,{0954B9AC-3079-473A-B231-AC6AC5422413},Ward 5,2012,2012,"POLYGON ((-76.99727251536488 38.91124895701469, -76.99664956091959 38.91146541521783, -76.996306..."
4,75162,3rd & M and Park Inn Associates LP,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=12-14,"Official Digital Zoning Map of July 1, 2010, Approved 3/14/14",R-5-D,Final,12-14,0,0,{D3F439EE-565C-4E75-BE94-5E922D8EC9C1},Ward 6,2019,2019,"(POLYGON ((-77.01538352279526 38.87774731884134, -77.01538340753703 38.87720607471513, -77.01592..."
5,75163,Prospect Joint Venture,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=85-8,"Official Digital Zoning Map of July 1, 2010",R-5-A,Final,85-8,0,0,{7281753A-8C87-427E-B290-FD5CC7299C5E},Ward 3,1985,1985,"POLYGON ((-77.05165473759811 38.93657780882258, -77.05164742961004 38.93655833892067, -77.051640..."
6,75164,Unified Communications Center,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=01-36,"Official Digital Zoning Map of July 1, 2010",C-2-B,Final,01-36,0,0,{CA2802A8-AE4C-4580-9C1E-05356E581877},Ward 8,2010,2010,"POLYGON ((-76.99367258295386 38.85433597906242, -76.99356059081177 38.85284743367416, -76.995811..."
7,75165,Matthews Memorial,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=08-25,"Official Digital Zoning Map of July 1, 2010",C-2-A,Final,08-25,0,0,{CC325C3B-83FC-4CC3-B528-C94CE258E127},Ward 8,2009,2009,"POLYGON ((-76.99450772873712 38.85733070854086, -76.99458188952184 38.85709882271153, -76.995684..."
8,75582,Nehemiah Homes at Fort Dupont,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=01-12,"Official Digital Zoning Map of July 1, 2010",None,Final,01-12,0,0,{A2848782-6E12-4AA2-B2B7-41E0471EDD34},Ward 7,2001,2001,"(POLYGON ((-76.94835414985999 38.88799472401496, -76.94808095550476 38.88772111014288, -76.94795..."
9,75583,Boston Properties II,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=87-18,"Official Digital Zoning Map of July 1, 2010",CR,Final,87-18,0,0,{2D661566-E6EA-498A-9DE0-26B61940093E},Ward 2,1988,1988,"POLYGON ((-77.05244449796034 38.90714705139236, -77.05244444792962 38.90679285039369, -77.052783..."


In [3]:
census = pd.read_csv('../data/open_data/csv/Census_Block_Groups__2010.csv')

In [4]:
aff = pd.read_csv('../data/open_data/csv/Affordable_Housing.csv')

In [5]:
aff['geometry'] = aff.apply(lambda row: Point(row.X, row.Y), axis=1)

In [6]:
aff = gpd.GeoDataFrame(aff, crs={'init' :'epsg:4326'})

In [7]:
evict_csv = pd.read_csv('../data/eviction_lab/csv/block-groups.csv')

In [8]:
evict_gis = gpd.read_file('../data/eviction_lab/gis/block-groups.geojson')

In [9]:
evict_csv.GEOID = evict_csv.GEOID.astype(str)

In [10]:
evict_gis.GEOID = evict_gis.GEOID.astype(str)

In [11]:
evict0 = evict_gis[['GEOID','geometry']].merge(evict_csv, on='GEOID', how='left')

In [12]:
evict0 = evict0.loc[evict0.year == 2016]

In [14]:
evict0

,GEOID,geometry,year,name,parent-location,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,median-property-value,rent-burden,pct-white,pct-af-am,pct-hispanic,pct-am-ind,pct-asian,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
16,110010012004,"(POLYGON ((-77.07888800000001 38.946504, -77.07865 38.946617, -77.078469 38.946643, -77.07841999...",2016,12.4,"District of Columbia, District of Columbia",1405.0,0.00,188.0,30.38,1442.0,149625.0,881500.0,33.4,82.06,0.93,4.70,0.00,11.25,0.0,1.07,0.00,8.0,0.0,0.00,4.26,1,0,0
33,110010013011,"(POLYGON ((-77.042253 38.96035, -77.040616 38.959672, -77.04222900000001 38.958067, -77.042394 3...",2016,13.01.1,"District of Columbia, District of Columbia",1771.0,8.04,517.0,67.50,2126.0,129829.0,1564700.0,27.7,81.48,6.66,4.74,0.00,5.99,0.0,1.13,0.00,20.0,2.0,0.39,3.87,1,0,0
50,110010013013,"(POLYGON ((-77.061632 38.951849, -77.061639 38.949679, -77.066714 38.949683, -77.069425 38.95433...",2016,13.01.3,"District of Columbia, District of Columbia",1568.0,0.00,884.0,81.92,1701.0,103182.0,1375000.0,23.2,69.64,14.99,8.04,0.00,5.99,0.0,1.34,0.00,30.0,4.0,0.45,3.39,1,0,0
67,110010013021,"(POLYGON ((-77.056712 38.94789, -77.056456 38.94695, -77.05623 38.946297, -77.055262 38.943509, ...",2016,13.02.1,"District of Columbia, District of Columbia",2409.0,8.57,916.0,56.37,2152.0,92345.0,370800.0,32.0,60.86,7.31,10.75,0.00,17.35,0.0,2.62,1.12,58.0,5.0,0.55,6.33,1,0,0
84,110010013022,"(POLYGON ((-77.055716 38.930755, -77.05677 38.93257, -77.059243 38.936824, -77.06151699999999 38...",2016,13.02.2,"District of Columbia, District of Columbia",4549.0,0.00,2370.0,79.90,1731.0,102976.0,379500.0,27.0,76.24,8.07,10.86,0.00,3.08,0.0,1.76,0.00,45.0,11.0,0.46,1.90,1,0,0
101,110010013023,"(POLYGON ((-77.04994000000001 38.944295, -77.051303 38.941752, -77.051638 38.939726, -77.052696 ...",2016,13.02.3,"District of Columbia, District of Columbia",633.0,0.00,384.0,50.48,1768.0,118107.0,1041700.0,36.8,69.19,0.00,7.90,0.00,22.91,0.0,0.00,0.00,24.0,7.0,1.82,6.25,1,0,0
118,110010014012,"(POLYGON ((-77.07743669820709 38.9673218177274, -77.0771138374774 38.96757003193859, -77.0716409...",2016,14.01.2,"District of Columbia, District of Columbia",1160.0,0.00,41.0,6.60,2875.0,214063.0,990000.0,22.5,92.59,0.09,2.93,0.00,1.38,0.0,1.90,1.12,1.0,0.0,0.00,2.44,1,0,0
135,110010014022,"(POLYGON ((-77.070831 38.961013, -77.066238 38.961013, -77.056201 38.961013, -77.056005 38.95892...",2016,14.02.2,"District of Columbia, District of Columbia",887.0,0.00,195.0,39.96,2171.0,152778.0,839000.0,37.5,80.72,11.16,2.25,0.00,3.49,0.0,2.37,0.00,2.0,0.0,0.00,1.03,1,0,0
152,110010014023,"(POLYGON ((-77.070831 38.961013, -77.067543 38.959388, -77.066919 38.959001, -77.06603699999999 ...",2016,14.02.3,"District of Columbia, District of Columbia",1557.0,14.55,448.0,62.01,1116.0,79063.0,689900.0,29.3,48.04,15.35,16.25,1.35,13.81,0.0,2.57,2.63,30.0,8.0,1.79,6.70,1,0,0
169,110010015001,"(POLYGON ((-77.05166250431029 38.9870923275862, -77.05177 38.986949, -77.05004099999999 38.98606...",2016,15.1,"District of Columbia, District of Columbia",1654.0,0.00,28.0,5.62,NaN,210714.0,932100.0,NaN,59.92,22.97,7.19,0.00,4.29,0.0,5.62,0.00,1.0,0.0,0.00,3.57,1,0,0


In [62]:
aff

,X,Y,OBJECTID,MAR_WARD,ADDRESS,PROJECT_NAME,STATUS_PUBLIC,AGENCY_CALCULATED,TOTAL_AFFORDABLE_UNITS,LATITUDE,LONGITUDE,AFFORDABLE_UNITS_AT_0_30_AMI,AFFORDABLE_UNITS_AT_31_50_AMI,AFFORDABLE_UNITS_AT_51_60_AMI,AFFORDABLE_UNITS_AT_61_80_AMI,AFFORDABLE_UNITS_AT_81_AMI,CASE_ID,ADDRESS_ID,XCOORD,YCOORD,FULLADDRESS,GIS_LAST_MOD_DTTM,geometry
0,-77.033608,38.937742,1556,Ward 4,"1410 Quincy Street Northwest, Washington, District of Columbia 20011",1408-1410 Quincy St NW,Under Construction,DHCD,1,38.937734,-77.033606,0,0,1,0,0,NaN,258989.0,397086.450000,141094.680000,1410 QUINCY STREET NW,2019-12-30T04:00:25.000Z,POINT (-77.033608040074 38.93774225671439)
1,-77.039751,38.915825,1557,NaN,"1729 T Street Northwest, Washington, District of Columbia 20009",1729 T St NW,Under Construction,DHCD,1,NaN,NaN,0,0,1,0,0,NaN,241407.0,396552.840000,138661.890000,1729 T STREET NW,2019-12-30T04:00:25.000Z,POINT (-77.0397506368645 38.91582498578156)
2,-77.034769,38.934611,1560,Ward 1,"1447 Oak Street Northwest, Washington, District of Columbia 20010",Rhode Island Condos,Under Construction,DHCD,2,38.934603,-77.034767,0,0,2,0,0,NaN,NaN,396985.639048,140747.098001,NaN,2019-12-30T04:00:25.000Z,POINT (-77.03476929648031 38.93461079384309)
3,-76.980438,38.869679,1561,Ward 8,"1704 R Street Southeast, Washington, District of Columbia 20020",1704 R St SE,Completed 2015 to Date,DHCD,1,38.869672,-76.980436,0,0,1,0,0,NaN,57916.0,401697.780000,133538.800000,1704 R STREET SE,2019-12-30T04:00:25.000Z,POINT (-76.98043825795355 38.86967941084557)
4,-76.974973,38.929333,1562,Ward 5,"2027 Rhode Island Avenue Northeast, Washington, District of Columbia 20018",2027 Rhode Island Ave,Under Construction,DHCD,6,38.929325,-76.974971,0,0,0,6,0,NaN,NaN,402170.219637,140160.917751,NaN,2019-12-30T04:00:25.000Z,POINT (-76.97497327753194 38.92933279501001)
5,-77.020661,38.915280,1563,Ward 6,"618 T Street Northwest, Washington, District of Columbia 20001",The Shaw,Under Construction,DHCD,5,38.915386,-77.020806,0,0,0,5,0,NaN,238431.0,398208.340000,138600.870000,618 T STREET NW,2019-12-30T04:00:25.000Z,POINT (-77.02066131607506 38.91528023213611)
6,-77.023189,38.956558,1564,Ward 4,"721 Kennedy Street Northwest, Washington, District of Columbia 20011",WSH 7K,Under Construction,DHCD,1,38.956550,-77.023187,0,0,0,1,0,NaN,250280.0,397990.300000,143183.080000,721 KENNEDY STREET NW,2019-12-30T04:00:25.000Z,POINT (-77.02318888466527 38.95655768321879)
7,-76.966708,38.933317,1565,Ward 5,"2911 Rhode Island Avenue Northeast, Washington, District of Columbia 20018",The Shirley,Under Construction,DHCD,13,38.933309,-76.966705,0,6,7,0,0,NaN,43463.0,402886.750000,140603.390000,2911 RHODE ISLAND AVENUE NE,2019-12-30T04:00:25.000Z,POINT (-76.96670771724307 38.93331665965651)
8,-76.987578,38.924711,1566,Ward 5,"2711 13th Street Northeast, Washington, District of Columbia 20017",Rhode Island Ave,Under Construction,DHCD,4,38.924704,-76.987576,0,0,4,0,0,NaN,305797.0,401077.320000,139647.670000,2711 13TH STREET NE,2019-12-30T04:00:25.000Z,POINT (-76.98757842598094 38.92471131644334)
9,-77.032483,38.918996,1567,Ward 1,"1400 W Street Northwest, Washington, District of Columbia 20009",1400 W ST NW,Under Construction,DHCD,20,38.918988,-77.032481,0,0,20,0,0,NaN,NaN,397183.222566,139013.626426,NaN,2019-12-30T04:00:25.000Z,POINT (-77.03248329481296 38.91899579078174)


In [75]:
puds.sindex
aff.sindex

In [30]:
puds_aff = gpd.sjoin(puds, aff, op='intersects', how='left')

In [31]:
puds_aff.loc[~puds_aff.PROJECT_NAME.isna()]

,OBJECTID_left,PUD_NAME,PUD_WEB_UR,PUD_CHANGE,PUD_ZONING,PUD_STATUS,PUD_CASE_N,SHAPEAREA,SHAPELEN,GLOBALID,WARD,year_scrap,year,geometry,index_right,X,Y,OBJECTID_right,MAR_WARD,ADDRESS,PROJECT_NAME,STATUS_PUBLIC,AGENCY_CALCULATED,TOTAL_AFFORDABLE_UNITS,LATITUDE,LONGITUDE,AFFORDABLE_UNITS_AT_0_30_AMI,AFFORDABLE_UNITS_AT_31_50_AMI,AFFORDABLE_UNITS_AT_51_60_AMI,AFFORDABLE_UNITS_AT_61_80_AMI,AFFORDABLE_UNITS_AT_81_AMI,CASE_ID,ADDRESS_ID,XCOORD,YCOORD,FULLADDRESS,GIS_LAST_MOD_DTTM
4,75162,3rd & M and Park Inn Associates LP,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=12-14,"Official Digital Zoning Map of July 1, 2010, Approved 3/14/14",R-5-D,Final,12-14,0,0,{D3F439EE-565C-4E75-BE94-5E922D8EC9C1},Ward 6,2019,2019,"(POLYGON ((-77.01538352279526 38.87774731884134, -77.01538340753703 38.87720607471513, -77.01592...",378.0,-77.015616,38.876760,1167.0,Ward 6,"301 M Street Southwest, Washington, District of Columbia 20024",301 M,Completed 2015 to Date,DHCD OP,15.0,38.876752,-77.015614,0.0,0.0,0.0,15.0,0.0,12-14/12-14A,312585.0,398645.170000,134324.710000,301 M STREET SW,2019-12-30T04:00:25.000Z
24,75166,Sheridan Redevelopment LLC & DCHA,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=07-35C,Modification 9/10/12,"R-5-B, C-2-A",Final,07-35C,0,0,{0C0FFE2C-8B69-45FB-BB39-BFC175759EF8},Ward 8,2012,2012,"(POLYGON ((-76.99156800915893 38.85736031346332, -76.99156141139086 38.85719293360246, -76.99236...",62.0,-76.992548,38.860224,97.0,Ward 8,"2500 Sayles Place Southeast, Washington, District of Columbia 20020",City Homes at Sheridan Station 2c,Completed 2015 to Date,DCHA,20.0,38.860216,-76.992546,0.0,0.0,20.0,0.0,0.0,NaN,308348.0,400646.940000,132488.990000,2500 SAYLES PLACE SE,2019-12-30T04:00:25.000Z
26,75168,Parcel Seven Associates LLC,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=10-03B,ZC Case 10-03B,None,Final,10-03B,0,0,{2513A84A-FB1E-4ED8-8676-121E5D965D18},Ward 6,None,2018,"POLYGON ((-76.99478742700059 38.90008376236103, -76.9927848489031 38.90008365587951, -76.9927848...",358.0,-76.993730,38.900043,1075.0,Ward 6,"901 H Street Northeast, Washington, District of Columbia 20002",Avec,Under Construction,DHCD OP,32.0,38.900035,-76.993727,0.0,0.0,0.0,32.0,0.0,10-03,289048.0,400544.120000,136909.230000,901 H STREET NE,2019-12-30T04:00:25.000Z
27,75521,Stanton-Eastbanc LLC,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=11-24,Case # 11-24,C-2-B,Final,11-24,0,0,{50F642D4-135C-40A3-A288-40D7D4105098},Ward 6,2012,2012,"POLYGON ((-76.9960191362397 38.8862595958395, -76.99560107399716 38.88625958085016, -76.99556534...",79.0,-76.995404,38.885860,142.0,Ward 6,"777 C St SE, Washington, District of Columbia 20003",Hine Junior High School (North LIHTC Residential),Completed 2015 to Date,DMPED DCHFA OP,46.0,38.885134,-76.995929,5.0,0.0,29.0,12.0,0.0,11-24,311940.0,400398.920000,135334.820000,777 C STREET SE,2019-12-30T04:00:25.000Z
28,75522,Jemals Babes,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=10-23,Case # 10-23,C-3-A,Final,10-23,0,0,{B6A3E99F-9C83-4A65-8BED-E86B68C75E54},Ward 3,2012,2012,"POLYGON ((-77.08123878944505 38.95015670271377, -77.08113330149931 38.95015677597164, -77.081027...",150.0,-77.080846,38.949980,410.0,Ward 3,"4600 Wisconsin Avenue, NW, Washington, District of Columbia",Tenley View,Completed 2015 to Date,DHCD OP,6.0,38.949947,-77.080827,0.0,0.0,0.0,6.0,0.0,10-23,264444.0,392992.220000,142455.740000,4600 WISCONSIN AVENUE NW,2019-12-30T04:00:25.000Z
29,75523,Capper/Carrollsburg Venture,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=03-12X/03-13X,"Official Digital Zoning Map of July 1, 2010",CR,Final,03-12X/03-13X,0,0,{D4027031-CE3F-48DD-93B7-A3785A25A197},Ward 6,None,2019,"(POLYGON ((-77.00220306544236 38.87661323607642, -77.0027981784101 38.87661322332325, -77.003040...",167.0,-77.002512,38.877202,328.0,Ward 6,"1100 2nd Place Southeast, Washington, District of Columbia 20003",The Harlow (Square 769N),Completed 

In [32]:
puds_aff.to_file("../data/viz/puds_aff.shp")

In [41]:
puds_aff.columns = ['OBJECTID_PUD', 'PUD_NAME', 'PUD_WEB_UR', 'PUD_CHANGE', 'PUD_ZONING',
       'PUD_STATUS', 'PUD_CASE_N', 'SHAPEAREA', 'SHAPELEN', 'GLOBALID', 'WARD',
       'year_scrap', 'year', 'geometry', 'index_right', 'X', 'Y',
       'OBJECTID_Aff_Housing', 'MAR_WARD', 'ADDRESS', 'PROJECT_NAME',
       'STATUS_PUBLIC', 'AGENCY_CALCULATED', 'TOTAL_AFFORDABLE_UNITS',
       'LATITUDE', 'LONGITUDE', 'AFFORDABLE_UNITS_AT_0_30_AMI',
       'AFFORDABLE_UNITS_AT_31_50_AMI', 'AFFORDABLE_UNITS_AT_51_60_AMI',
       'AFFORDABLE_UNITS_AT_61_80_AMI', 'AFFORDABLE_UNITS_AT_81_AMI',
       'CASE_ID', 'ADDRESS_ID', 'XCOORD', 'YCOORD', 'FULLADDRESS',
       'GIS_LAST_MOD_DTTM']

In [43]:
puds_aff.columns

Index(['OBJECTID_PUD', 'PUD_NAME', 'PUD_WEB_UR', 'PUD_CHANGE', 'PUD_ZONING',
       'PUD_STATUS', 'PUD_CASE_N', 'SHAPEAREA', 'SHAPELEN', 'GLOBALID', 'WARD',
       'year_scrap', 'year', 'geometry', 'index_right', 'X', 'Y',
       'OBJECTID_Aff_Housing', 'MAR_WARD', 'ADDRESS', 'PROJECT_NAME',
       'STATUS_PUBLIC', 'AGENCY_CALCULATED', 'TOTAL_AFFORDABLE_UNITS',
       'LATITUDE', 'LONGITUDE', 'AFFORDABLE_UNITS_AT_0_30_AMI',
       'AFFORDABLE_UNITS_AT_31_50_AMI', 'AFFORDABLE_UNITS_AT_51_60_AMI',
       'AFFORDABLE_UNITS_AT_61_80_AMI', 'AFFORDABLE_UNITS_AT_81_AMI',
       'CASE_ID', 'ADDRESS_ID', 'XCOORD', 'YCOORD', 'FULLADDRESS',
       'GIS_LAST_MOD_DTTM'],
      dtype='object')

In [46]:
puds_aff = puds_aff.drop(['index_right'], axis=1)

In [33]:
evict0

,GEOID,geometry,year,name,parent-location,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,median-property-value,rent-burden,pct-white,pct-af-am,pct-hispanic,pct-am-ind,pct-asian,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
16,110010012004,"(POLYGON ((-77.07888800000001 38.946504, -77.07865 38.946617, -77.078469 38.946643, -77.07841999...",2016,12.4,"District of Columbia, District of Columbia",1405.0,0.00,188.0,30.38,1442.0,149625.0,881500.0,33.4,82.06,0.93,4.70,0.00,11.25,0.0,1.07,0.00,8.0,0.0,0.00,4.26,1,0,0
33,110010013011,"(POLYGON ((-77.042253 38.96035, -77.040616 38.959672, -77.04222900000001 38.958067, -77.042394 3...",2016,13.01.1,"District of Columbia, District of Columbia",1771.0,8.04,517.0,67.50,2126.0,129829.0,1564700.0,27.7,81.48,6.66,4.74,0.00,5.99,0.0,1.13,0.00,20.0,2.0,0.39,3.87,1,0,0
50,110010013013,"(POLYGON ((-77.061632 38.951849, -77.061639 38.949679, -77.066714 38.949683, -77.069425 38.95433...",2016,13.01.3,"District of Columbia, District of Columbia",1568.0,0.00,884.0,81.92,1701.0,103182.0,1375000.0,23.2,69.64,14.99,8.04,0.00,5.99,0.0,1.34,0.00,30.0,4.0,0.45,3.39,1,0,0
67,110010013021,"(POLYGON ((-77.056712 38.94789, -77.056456 38.94695, -77.05623 38.946297, -77.055262 38.943509, ...",2016,13.02.1,"District of Columbia, District of Columbia",2409.0,8.57,916.0,56.37,2152.0,92345.0,370800.0,32.0,60.86,7.31,10.75,0.00,17.35,0.0,2.62,1.12,58.0,5.0,0.55,6.33,1,0,0
84,110010013022,"(POLYGON ((-77.055716 38.930755, -77.05677 38.93257, -77.059243 38.936824, -77.06151699999999 38...",2016,13.02.2,"District of Columbia, District of Columbia",4549.0,0.00,2370.0,79.90,1731.0,102976.0,379500.0,27.0,76.24,8.07,10.86,0.00,3.08,0.0,1.76,0.00,45.0,11.0,0.46,1.90,1,0,0
101,110010013023,"(POLYGON ((-77.04994000000001 38.944295, -77.051303 38.941752, -77.051638 38.939726, -77.052696 ...",2016,13.02.3,"District of Columbia, District of Columbia",633.0,0.00,384.0,50.48,1768.0,118107.0,1041700.0,36.8,69.19,0.00,7.90,0.00,22.91,0.0,0.00,0.00,24.0,7.0,1.82,6.25,1,0,0
118,110010014012,"(POLYGON ((-77.07743669820709 38.9673218177274, -77.0771138374774 38.96757003193859, -77.0716409...",2016,14.01.2,"District of Columbia, District of Columbia",1160.0,0.00,41.0,6.60,2875.0,214063.0,990000.0,22.5,92.59,0.09,2.93,0.00,1.38,0.0,1.90,1.12,1.0,0.0,0.00,2.44,1,0,0
135,110010014022,"(POLYGON ((-77.070831 38.961013, -77.066238 38.961013, -77.056201 38.961013, -77.056005 38.95892...",2016,14.02.2,"District of Columbia, District of Columbia",887.0,0.00,195.0,39.96,2171.0,152778.0,839000.0,37.5,80.72,11.16,2.25,0.00,3.49,0.0,2.37,0.00,2.0,0.0,0.00,1.03,1,0,0
152,110010014023,"(POLYGON ((-77.070831 38.961013, -77.067543 38.959388, -77.066919 38.959001, -77.06603699999999 ...",2016,14.02.3,"District of Columbia, District of Columbia",1557.0,14.55,448.0,62.01,1116.0,79063.0,689900.0,29.3,48.04,15.35,16.25,1.35,13.81,0.0,2.57,2.63,30.0,8.0,1.79,6.70,1,0,0
169,110010015001,"(POLYGON ((-77.05166250431029 38.9870923275862, -77.05177 38.986949, -77.05004099999999 38.98606...",2016,15.1,"District of Columbia, District of Columbia",1654.0,0.00,28.0,5.62,NaN,210714.0,932100.0,NaN,59.92,22.97,7.19,0.00,4.29,0.0,5.62,0.00,1.0,0.0,0.00,3.57,1,0,0


In [78]:
puds_info = gpd.sjoin(evict0, puds_aff, op='intersects', how='right')

In [79]:
puds_info = puds_info.drop(['index_left', 'year_x', 'name', 'parent-location', 'imputed', 'subbed'], axis=1)

In [80]:
puds_xwalk = pd.read_csv('../data/grassroots_dc0/csv/zoning_crosswalk.csv')

In [70]:
puds_xwalk

,Zone,Count,Zone_Cat
0,C-3-C,48,Commercial
1,None,36,Other/Mixed Use
2,C-2-B,21,Commercial
3,R-5-B,17,Residential
4,C-4,14,Commercial
5,R-5-A,13,Residential
6,C-3-A,12,Commercial
7,"C-3-A, CR",10,Commercial
8,C-3-B,9,Commercial
9,CR,8,Commercial


In [81]:
puds_info = puds_info.merge(puds_xwalk[['Zone', 'Zone_Cat']], how='left', left_on='PUD_ZONING', right_on=puds_xwalk['Zone'])

In [75]:
puds_info.columns

Index(['GEOID', 'population', 'poverty-rate', 'renter-occupied-households',
       'pct-renter-occupied', 'median-gross-rent', 'median-household-income',
       'median-property-value', 'rent-burden', 'pct-white', 'pct-af-am',
       'pct-hispanic', 'pct-am-ind', 'pct-asian', 'pct-nh-pi', 'pct-multiple',
       'pct-other', 'eviction-filings', 'evictions', 'eviction-rate',
       'eviction-filing-rate', 'low-flag', 'OBJECTID_PUD', 'PUD_NAME',
       'PUD_WEB_UR', 'PUD_CHANGE', 'PUD_ZONING', 'PUD_STATUS', 'PUD_CASE_N',
       'SHAPEAREA', 'SHAPELEN', 'GLOBALID', 'WARD', 'year_scrap', 'year_y',
       'geometry', 'X', 'Y', 'OBJECTID_Aff_Housing', 'MAR_WARD', 'ADDRESS',
       'PROJECT_NAME', 'STATUS_PUBLIC', 'AGENCY_CALCULATED',
       'TOTAL_AFFORDABLE_UNITS', 'LATITUDE', 'LONGITUDE',
       'AFFORDABLE_UNITS_AT_0_30_AMI', 'AFFORDABLE_UNITS_AT_31_50_AMI',
       'AFFORDABLE_UNITS_AT_51_60_AMI', 'AFFORDABLE_UNITS_AT_61_80_AMI',
       'AFFORDABLE_UNITS_AT_81_AMI', 'CASE_ID', 'ADDRESS_ID'

In [83]:
puds_info = puds_info[['GEOID', 'population', 'poverty-rate', 'renter-occupied-households',
       'pct-renter-occupied', 'median-gross-rent', 'median-household-income',
       'median-property-value', 'rent-burden', 'pct-white', 'pct-af-am',
       'pct-hispanic', 'pct-am-ind', 'pct-asian', 'pct-nh-pi', 'pct-multiple',
       'pct-other', 'eviction-filings', 'evictions', 'eviction-rate',
       'eviction-filing-rate', 'low-flag', 'OBJECTID_PUD', 'PUD_NAME',
       'PUD_WEB_UR', 'PUD_CHANGE', 'PUD_ZONING', 'Zone_Cat', 'PUD_STATUS', 'PUD_CASE_N',
       'SHAPEAREA', 'SHAPELEN', 'GLOBALID', 'WARD', 'year_scrap', 'year_y',
       'geometry', 'X', 'Y', 'OBJECTID_Aff_Housing', 'MAR_WARD', 'ADDRESS',
       'PROJECT_NAME', 'STATUS_PUBLIC', 'AGENCY_CALCULATED',
       'TOTAL_AFFORDABLE_UNITS', 'LATITUDE', 'LONGITUDE',
       'AFFORDABLE_UNITS_AT_0_30_AMI', 'AFFORDABLE_UNITS_AT_31_50_AMI',
       'AFFORDABLE_UNITS_AT_51_60_AMI', 'AFFORDABLE_UNITS_AT_61_80_AMI',
       'AFFORDABLE_UNITS_AT_81_AMI', 'CASE_ID', 'ADDRESS_ID', 'XCOORD',
       'YCOORD', 'FULLADDRESS', 'GIS_LAST_MOD_DTTM']]

In [86]:
puds_info.rename(columns={'year_y':'year'}, inplace=True)

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [87]:
puds_info

,GEOID,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,median-property-value,rent-burden,pct-white,pct-af-am,pct-hispanic,pct-am-ind,pct-asian,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,OBJECTID_PUD,PUD_NAME,PUD_WEB_UR,PUD_CHANGE,PUD_ZONING,Zone_Cat,PUD_STATUS,PUD_CASE_N,SHAPEAREA,SHAPELEN,GLOBALID,WARD,year_scrap,year,geometry,X,Y,OBJECTID_Aff_Housing,MAR_WARD,ADDRESS,PROJECT_NAME,STATUS_PUBLIC,AGENCY_CALCULATED,TOTAL_AFFORDABLE_UNITS,LATITUDE,LONGITUDE,AFFORDABLE_UNITS_AT_0_30_AMI,AFFORDABLE_UNITS_AT_31_50_AMI,AFFORDABLE_UNITS_AT_51_60_AMI,AFFORDABLE_UNITS_AT_61_80_AMI,AFFORDABLE_UNITS_AT_81_AMI,CASE_ID,ADDRESS_ID,XCOORD,YCOORD,FULLADDRESS,GIS_LAST_MOD_DTTM
0,110010092031,1159.0,30.40,155.0,45.66,427.0,NaN,423600.0,50.0,8.11,82.66,4.83,0.00,0.00,0.00,4.40,0.00,32.0,5.0,3.23,20.65,1,75158,4L Associates Joint Venture,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=80-17,"Official Digital Zoning Map of July 1, 2010",R-5-A,Residential,Final,80-17,0,0,{95450152-37F7-41C5-B147-0C2719D9EAFA},Ward 5,1981,1981,"POLYGON ((-77.00471574325915 38.92462196572229, -77.00473300809519 38.92468044427013, -77.004756...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110010092011,2261.0,7.76,540.0,63.41,1229.0,52734.0,470700.0,30.8,34.32,52.45,7.52,0.00,1.42,0.00,4.29,0.00,170.0,17.0,3.15,31.48,1,75159,Bozutto Group,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=08-24C/04-25,PUD Modification July 2018,"C-2-B, R-5-B",Other/Mixed Use,Final,08-24C/04-25,0,0,{1FF4275D-29FF-4D38-8CFE-D7119010FAF5},Ward 5,2018,2018,"(POLYGON ((-76.99633034837495 38.9303615341334, -76.99688238904335 38.93036155015298, -76.998094...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110010058002,1781.0,10.44,813.0,58.05,2171.0,111023.0,475700.0,30.1,53.90,9.83,5.61,0.00,27.85,0.00,2.47,0.34,14.0,6.0,0.74,1.72,1,75160,National Academy of Sciences - Fifth Street,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=01-34,"Official Digital Zoning Map of July 1, 2010",C-3-C,Commercial,Final,01-34,0,0,{F5A40354-52E1-486B-A268-201E28D938FF},Ward 2,2002,2002,"POLYGON ((-77.01907499933805 38.89721344569826, -77.01907474363878 38.89625726176668, -77.019730...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110010088031,2148.0,48.50,469.0,82.26,1044.0,20313.0,224500.0,38.0,23.23,59.92,10.01,0.42,3.31,0.00,2.84,0.28,101.0,23.0,4.90,21.54,1,75161,Baywood Hotels,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=11-25,Case # 11-25,C-3-C,Commercial,Final,11-25,0,0,{0954B9AC-3079-473A-B231-AC6AC5422413},Ward 5,2012,2012,"POLYGON ((-76.99727251536488 38.91124895701469, -76.99664956091959 38.91146541521783, -76.996306...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110010102002,1493.0,0.00,1043.0,83.06,1750.0,91667.0,370200.0,24.3,38.98,40.52,9.98,0.00,6.23,0.00,4.29,0.00,200.0,11.0,1.05,19.18,1,75162,3rd & M and Park Inn Associates LP,https://app.dcoz.dc.gov/Content/Search/ViewCaseReport.aspx?case_id=12-14,"Official Digital Zoning Map of July 1, 2010, Approved 3/14/14",R-5-D,Residential,Final,12-14,0,0,{D3F439EE-565C-4E75-BE94-5E922D8EC9C1},Ward 6,2019,2019,"(POLYGON ((-77.01538352279526 38.87774731884134, -77.01538340753703 38.87720607471513, -77.01592...",-77.015616,38.876760,1167.0,Ward 6,"301 M Street Southwest, Washington, District of Columbia 20024",301 M,Completed 2015 to Date,DHCD OP,15.0,38.876752,-77.015614,0.0,0.0,0.0,15.0,0.0,12-14/12-14A,312585.0,398645.170000,134324.710000,301 M STREET SW,2019-12-30T04:00:25.000Z
5,110010013022,4549.0,0.00,2370.0,79.90,1731.0,102976.0,379500.0,27.0,76.24,8.07,10.86,0.00,3.08,0.00,1.76,0.00,45.0,11.0,0.46,1.90,1,75163,Prospect Joint Venture,https://app.dcoz.dc.gov/Content/Search/

In [88]:
puds_info.to_file("../data/viz/puds_info.shp")

In [97]:
puds_info.year = puds_info.year.astype(int)

In [98]:
puds_11_12 = puds_info.loc[(puds_info.year == 2011) | (puds_info.year == 2012)]
puds_13_14 = puds_info.loc[(puds_info.year == 2013) | (puds_info.year == 2014)]
puds_15_16 = puds_info.loc[(puds_info.year == 2015) | (puds_info.year == 2016)]

In [104]:
no_puds = evict0[(~evict0.GEOID.isin(puds_info.GEOID))]

In [107]:
print(f"No PUDs mean eviction rate:      {round(no_puds['eviction-rate'].mean(),3)}%, n = {no_puds.shape[0]}")
print(f"2011-12 PUDs mean eviction rate: {round(puds_11_12['eviction-rate'].mean(),3)}%, n = {puds_11_12.shape[0]}")
print(f"2013-14 PUDs mean eviction rate: {round(puds_13_14['eviction-rate'].mean(),3)}%, n = {puds_13_14.shape[0]}")
print(f"2015-16 PUDs mean eviction rate: {round(puds_15_16['eviction-rate'].mean(),3)}%, n = {puds_15_16.shape[0]}")

No PUDs mean eviction rate:      2.425%, n = 295
2011-12 PUDs mean eviction rate: 2.917%, n = 23
2013-14 PUDs mean eviction rate: 2.422%, n = 25
2015-16 PUDs mean eviction rate: 3.567%, n = 42
